In [1]:
import pandas as pd
#Set it to None to display all columns/rows in the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows',100)

In [1]:
def print_csv(df):
    filename = input('Enter filename: ')
    try:
        df.to_csv('./{}.csv'.format(filename))
    except:
        print('Error')
        return
    print('{}.csv stored to the same directory as this file!'.format(filename))

In [4]:
def get_node_metadata(tocheck_id,map_id):
    url = 'https://api.graphika.com/maps/{}/nodes/{}'.format(map_id,tocheck_id)
#     print(url)
    r = requests.get(url, auth = (username,pswd))
    try:
        return pd.DataFrame([r.json()])
    except:
        print('**Permission denied for map {}**'.format(map_id))
        return 'Unknown'

In [ ]:
def get_map_counts(df):
    how_many_in_map = {}
    for mid in df.groupby('map_id').groups:
        how_many = df.groupby('map_id').get_group(mid).shape[0]
        how_many_in_map[mid] = [how_many,100*(how_many/len(df))]
        
    return pd.DataFrame.from_dict(how_many_in_map,orient='index',columns=['count','percent']).sort_values(['count'],ascending = False)

In [2]:
def get_map_name(map_id):
    url = 'https://api.graphika.com/maps/{}'.format(map_id)
    r = requests.get(url,auth=(username,pswd))
    try:
        return r.json()['name']
    except:
        print('**Permission denied for map {}**'.format(map_id))
        return 'Unknown'

In [ ]:
def get_map_ids(ids_to_check_list):
    check_for_tweet(ids_to_check_list[0])
    maps_dict = {}
    for an_id in ids_to_check_list:
        maps_dict[an_id] = check_for_tweet(an_id).time

    for k in maps_dict.keys():
        start_date = maps_dict[k].values[0]
        end_date = maps_dict[k].values[0] + np.timedelta64(7,'D')
        rtrs = get_rtrs(k)
        maps_dict[k] = get_map_counts(get_retweet_data('retweets',k,start_date,end_date))

    return maps_dict

In [3]:
def check_graphika_data(df,get_metadata = False):
    all_nodes = set(df.user_id.tolist())
    print('From this search, {} nodes were returned'.format(len(all_nodes)))
    result_node_df = pd.DataFrame()

    for node_id in all_nodes:
        result_node_df = result_node_df.append(get_hit('nodes',node_id,how_many = 10,get_groups = get_metadata,get_clusters = get_metadata,get_nodenames = get_metadata))

    map_summary_table = get_map_counts(result_node_df)
    top_maps = map_summary_table.index.tolist()[:3]
    top_names = [get_map_name(top_maps[0]),get_map_name(top_maps[1]),get_map_name(top_maps[2])]
    print('The map that have the most of these nodes are \n{} \n{} \nand {}'.format(top_names[0],top_names[1],top_names[2]))
    display(get_map_counts(result_node_df))
    
    if input('>> Would you like to save a table of which maps these nodes appear in? (y/n) \n') == 'y':
        print_csv(result_node_df.set_index('map_id'))
    return result_node_df.set_index('map_id')
    # current_df = get_hit('nodes','2972164989',how_many = 10)
    # set(current_df.map_id.tolist())
    # current_df = newtemp[newtemp.map_id == 1687]

In [ ]:
# def node_finder(node_id,get_groups = True,get_clusters = True,get_nodenames = True):
#     query = "SELECT * FROM \
#     \
#     ( \
#     SELECT \
#     * \
#     FROM \
#     hits_twitter_tweets \
#     JOIN \
#     map_nodes on map_nodes.node_id = hits_twitter_tweets.node_id \
#     WHERE \
#     hits_twitter_tweets.node_id = '{}' \
#     ) s;".format(node_id)
    
#     r = cur.execute(query)
#     hits = cur.fetchall()
#     hits_df = pd.DataFrame(hits, columns=["id","hit_value", "node_id", "message_id","hit_time","params","id_2","node_id2","map_id","cluster_id"])
    
#     hits_df["node_id"] = hits_df["node_id"].astype("str")
#     hits_df["message_id"] = hits_df["message_id"].astype("str")
#     hits_df["hit_type"] = 'tweets'
#     hits_df.drop(columns = ['node_id2','id_2','id','params'],inplace = True)
    
#     if get_groups:
#         groupname_values = hits_df.apply(groupname_lookup,axis = 1,result_type = 'reduce')
#         hits_df.insert(loc = hits_df.shape[1]-1,column = 'group_name', value = groupname_values)
        
#     if get_clusters:
#         clustername_values = hits_df.apply(clustername_lookup,axis = 1,result_type = 'reduce')
#         hits_df.insert(loc = hits_df.shape[1]-1,column = 'cluster_name', value = clustername_values)
        
#     if get_nodenames:
#         nodename_values = hits_df.apply(nodename_lookup,axis = 1,result_type = 'reduce')
#         hits_df.insert(loc = hits_df.shape[1]-1,column = 'node_name', value = nodename_values)

    
#     return hits_df